In [1]:
import pandas as pd 
import base64
import pyModeS as pms
df = pd.read_csv("/Users/hamzatriki/3ºProyectoDeDatos2/proyecto/1090_riddle/output.csv",sep = ";")
df = df.drop(columns="Unnamed: 2")
df

,ts_kafka,message
0,1733011203260,jUoZIupCiFgBPAjykJ4=
1,1733011203260,oAAUllilYmBIYPDDtec=
2,1733011203260,oAAZEMMJ5TEgP//kyUg=
3,1733011203260,jQIBK/gjAAIASbh2gt8=
4,1733011203260,XTRCE1QV9w==
...,...,...
5299995,1733125070354,qAAcMM9Z3y+/N+Jqk5Q=
5299996,1733125070363,oAAYv8zp2TA/9/9zeYc=
5299997,1733125070364,XUyvtCzrfw==
5299998,1733125070364,jTRDRFhD9lnm+bTyEwQ=


In [2]:
def encodeHex(b64):
    return base64.b64decode(b64).hex()

In [4]:
def getDownlink(hex):
    return pms.df(hex)

In [5]:
def msgIsCorrupted(hex):
    return (pms.crc(hex) != 0)

In [6]:
def getICAO(hex):
    return str(pms.common.icao(hex))

In [7]:
def getOnGround(hex):
    decimal_value =  pms.bin2int(pms.hex2bin(hex)[5:8]) 
    if decimal_value == 4:
        return 1
    elif decimal_value == 5:
        return 0
    else:
        return None
    

In [8]:
df["messageHex"] = df["message"].apply(encodeHex)
df["DL"] = df["messageHex"].apply(getDownlink)
df_11 = df[df["DL"] == 11]
df_11 =df_11.reset_index()


In [9]:
df_11.head()

,index,ts_kafka,message,messageHex,DL
0,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11
1,28,1733011203261,XVAtXkh/IA==,5d502d5e487f20,11
2,54,1733011206603,XjQjVWd01A==,5e3423556774d4,11
3,56,1733011206603,XUytpDAQpw==,5d4cada43010a7,11
4,64,1733011206603,XUnS7vEibg==,5d49d2eef1226e,11


In [11]:
df["ICAO"] = df["messageHex"].apply(getICAO)
df_17_18 =  df[(df["DL"] == 17) |((df["DL"] == 18))]
df_17_18 = df_17_18.reset_index()
df_17_18

,index,ts_kafka,message,messageHex,DL,ICAO
0,0,1733011203260,jUoZIupCiFgBPAjykJ4=,8d4a1922ea428858013c08f2909e,17,4a1922
1,3,1733011203260,jQIBK/gjAAIASbh2gt8=,8d02012bf82300020049b87682df,17,02012b
2,8,1733011203260,jTRCE5kJwZAgDCQK1nA=,8d3442139909c190200c240ad670,17,344213
3,9,1733011203260,jTRCE5kJwZAgDCQK1nA=,8d3442139909c190200c240ad670,17,344213
4,10,1733011203260,jUytpOEHCQAAAAAJsfk=,8d4cada4e107090000000009b1f9,17,4cada4
...,...,...,...,...,...,...
1987117,5299984,1733125070354,kDQjRigABki4f7DpsFQ=,9034234628000648b87fb0e9b054,18,342346
1987118,5299988,1733125070354,jUlSEepOOGABX4gy+go=,8d495211ea4e3860015f8832fa0a,17,495211
1987119,5299993,1733125070354,jTRkWJkMcaFYMBsLGiM=,8d346458990c71a158301b0b1a23,17,346458
1987120,5299998,1733125070364,jTRDRFhD9lnm+bTyEwQ=,8d3443445843f659e6f9b4f21304,17,344344


In [12]:
def getTypeCode(hex):
    return pms.common.typecode(hex)

In [13]:
def getCallSign(hex,typecode):
    if typecode >= 1 and typecode <= 4:
        return pms.decoder.adsb.callsign(hex)
    return None

In [14]:
df_17_18["TC"] = df_17_18["messageHex"].apply(getTypeCode)
df_17_18.head()

,index,ts_kafka,message,messageHex,DL,ICAO,TC
0,0,1733011203260,jUoZIupCiFgBPAjykJ4=,8d4a1922ea428858013c08f2909e,17,4a1922,29
1,3,1733011203260,jQIBK/gjAAIASbh2gt8=,8d02012bf82300020049b87682df,17,02012b,31
2,8,1733011203260,jTRCE5kJwZAgDCQK1nA=,8d3442139909c190200c240ad670,17,344213,19
3,9,1733011203260,jTRCE5kJwZAgDCQK1nA=,8d3442139909c190200c240ad670,17,344213,19
4,10,1733011203260,jUytpOEHCQAAAAAJsfk=,8d4cada4e107090000000009b1f9,17,4cada4,28


In [15]:
df_17_18["CS"] = df_17_18.apply(lambda row: getCallSign(row["messageHex"], row["TC"]), axis=1)
df_17_18 = df_17_18.dropna(subset=["CS"])
df_17_18

/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


,index,ts_kafka,message,messageHex,DL,ICAO,TC,CS
7,17,1733011203261,kDQjRBBRgxXDCCCkpe8=,9034234410518315c30820a4a5ef,18,342344,2,TXLU00__
11,31,1733011203261,kDQjRhBRgxXDKCBnVaI=,9034234610518315c328206755a2,18,342346,2,TXLU02__
19,44,1733011206603,kDQjSRBRgxXDWCBLilw=,9034234910518315c358204b8a5c,18,342349,2,TXLU05__
23,58,1733011206603,jDRBFRLfffdhAHFvW/Q=,8c34411512df7df76100716f5bf4,17,344115,2,7777XPA1
33,97,1733011207316,kDQjRRBRgxXDGCA6p80=,9034234510518315c318203aa7cd,18,342345,2,TXLU01__
...,...,...,...,...,...,...,...,...
1987056,5299847,1733125067179,jTRmEyNY8XPXPaASpSM=,8d3466132358f173d73da012a523,17,346613,4,VOE3536_
1987080,5299910,1733125069433,jUympCNJlLnRciD5jfc=,8d4ca6a4234994b9d17220f98df7,17,4ca6a4,4,RYR94WH_
1987084,5299918,1733125069433,kDQjSRBRgxXDWCBLilw=,9034234910518315c358204b8a5c,18,342349,2,TXLU05__
1987098,5299953,1733125069847,jTmlbyIxcfHciCBov+c=,8d39a56f223171f1dc882068bfe7,17,39a56f,4,LWG17H__


In [16]:
df_17_18.groupby(['ICAO',"CS"]).count().reset_index().sort_values(by=['CS']).head(10)

,ICAO,CS,index,ts_kafka,message,messageHex,DL,TC
648,344115,7777XPA1,1047,1047,1047,1047,1047,1047
5292,aa944a,AAL112__,25,25,25,25,25,25
5293,aa944a,AAL113__,1,1,1,1,1,1
5288,aa8315,AAL113__,1,1,1,1,1,1
5325,ab4ae9,AAL120__,1,1,1,1,1,1
5323,ab3c0d,AAL120__,1,1,1,1,1,1
5326,ab4ae9,AAL121__,18,18,18,18,18,18
5312,aad981,AAL37___,5,5,5,5,5,5
5306,aab812,AAL37___,5,5,5,5,5,5
5286,aa7439,AAL69___,33,33,33,33,33,33


In [17]:
df_counts = df_17_18.groupby(["ICAO", "CS"]).size().reset_index(name="count")
df_counts_sorted = df_counts.sort_values(by=["CS"]).head(10)
df_counts_sorted

,ICAO,CS,count
648,344115,7777XPA1,1047
5292,aa944a,AAL112__,25
5293,aa944a,AAL113__,1
5288,aa8315,AAL113__,1
5325,ab4ae9,AAL120__,1
5323,ab3c0d,AAL120__,1
5326,ab4ae9,AAL121__,18
5312,aad981,AAL37___,5
5306,aab812,AAL37___,5
5286,aa7439,AAL69___,33


In [18]:
df_17_18["timestamp"] =pd.to_datetime(df_17_18["ts_kafka"],unit='ms')
df_17_18.head()

/var/folders/kf/v4wgzcdn2k5_yn3_h219svlm0000gn/T/ipykernel_52558/2272512310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_17_18["timestamp"] =pd.to_datetime(df_17_18["ts_kafka"],unit='ms')


,index,ts_kafka,message,messageHex,DL,ICAO,TC,CS,timestamp
7,17,1733011203261,kDQjRBBRgxXDCCCkpe8=,9034234410518315c30820a4a5ef,18,342344,2,TXLU00__,2024-12-01 00:00:03.261
11,31,1733011203261,kDQjRhBRgxXDKCBnVaI=,9034234610518315c328206755a2,18,342346,2,TXLU02__,2024-12-01 00:00:03.261
19,44,1733011206603,kDQjSRBRgxXDWCBLilw=,9034234910518315c358204b8a5c,18,342349,2,TXLU05__,2024-12-01 00:00:06.603
23,58,1733011206603,jDRBFRLfffdhAHFvW/Q=,8c34411512df7df76100716f5bf4,17,344115,2,7777XPA1,2024-12-01 00:00:06.603
33,97,1733011207316,kDQjRRBRgxXDGCA6p80=,9034234510518315c318203aa7cd,18,342345,2,TXLU01__,2024-12-01 00:00:07.316


In [19]:
# Step 1: Sort Data
df_17_18 = df_17_18.sort_values(["ICAO", "CS", "timestamp"])

# Step 2: Compute time differences between consecutive messages
df_17_18["time_diff"] = df_17_18.groupby(["ICAO", "CS"])["timestamp"].diff()

# Step 3: Detect new flights (if time gap > 2 hours = 7200 seconds)
df_17_18["new_flight"] = (df_17_18["time_diff"] > pd.Timedelta(seconds=7200)).fillna(False)

# Step 4: Assign a unique Flight ID using cumulative sum
df_17_18["FLIGHT_ID"] = df_17_18.groupby(["ICAO", "CS"])["new_flight"].cumsum()

# Step 5: Group by ICAO, CS, and Flight ID to get min/max timestamps
df_callsigns = (
    df_17_18.groupby(["ICAO", "CS", "FLIGHT_ID"])["timestamp"]
    .agg(MIN_TIMESTAMP="min", MAX_TIMESTAMP="max")
    .reset_index()
)
df_callsigns

,ICAO,CS,FLIGHT_ID,MIN_TIMESTAMP,MAX_TIMESTAMP
0,000001,TALUD28_,0,2024-12-04 16:26:30.413,2024-12-04 16:32:29.696
1,000001,TALUD28_,1,2024-12-04 19:11:34.061,2024-12-04 19:19:52.507
2,000001,TALUD34_,0,2024-12-04 11:55:34.937,2024-12-04 12:08:57.597
3,010078,EGY1124_,0,2024-12-03 08:16:42.055,2024-12-03 08:19:32.103
4,0101ba,MSR754__,0,2024-12-01 13:21:31.690,2024-12-01 13:22:51.499
...,...,...,...,...,...
6952,e8046d,TAM8067_,0,2024-12-05 13:07:03.639,2024-12-05 13:07:03.639
6953,e8046d,TAM8068_,0,2024-12-05 06:30:44.882,2024-12-05 06:39:54.825
6954,e8046f,LAN9957_,0,2024-12-06 18:01:08.563,2024-12-06 18:06:32.385
6955,e80473,LAN714__,0,2024-12-02 12:10:37.432,2024-12-02 12:19:13.368


In [20]:
df_callsigns.groupby(["ICAO"])["FLIGHT_ID"].count().reset_index().sort_values(by=["FLIGHT_ID"],ascending=False).head(10)

,ICAO,FLIGHT_ID
172,342351,47
166,34234a,46
163,342347,46
103,34108f,46
171,342350,46
170,34234f,46
167,34234b,46
165,342349,46
164,342348,46
225,344115,46


In [93]:
"""df_callsigns = (
    df_17_18.groupby(["ICAO", "CS"])["timestamp"]
    .agg([("MIN_TIMESTAMP", "min"), ("MAX_TIMESTAMP", "max")])
    .reset_index()
)
df_callsigns
"""

,ICAO,CS,MIN_TIMESTAMP,MAX_TIMESTAMP
0,000001,TALUD28_,2024-12-04 16:26:30.413,2024-12-04 19:19:52.507
1,000001,TALUD34_,2024-12-04 11:55:34.937,2024-12-04 12:08:57.597
2,010078,EGY1124_,2024-12-03 08:16:42.055,2024-12-03 08:19:32.103
3,0101ba,MSR754__,2024-12-01 13:21:31.690,2024-12-01 13:22:51.499
4,0101be,MSR754__,2024-12-07 14:03:49.267,2024-12-07 14:09:29.752
...,...,...,...,...
5379,e8046d,TAM8067_,2024-12-05 13:07:03.639,2024-12-05 13:07:03.639
5380,e8046d,TAM8068_,2024-12-05 06:30:44.882,2024-12-05 06:39:54.825
5381,e8046f,LAN9957_,2024-12-06 18:01:08.563,2024-12-06 18:06:32.385
5382,e80473,LAN714__,2024-12-02 12:10:37.432,2024-12-02 12:19:13.368


In [21]:
df_11["ICAO"] = df_11["messageHex"].apply(getICAO)

In [22]:
df_11["OnGround"] = df_11["messageHex"].apply(getOnGround)
df_11 = df_11.dropna(subset=["OnGround"])

In [23]:
df_11['timestamp'] = pd.to_datetime(df_11['ts_kafka'], unit='ms')
df_11['formatted_date'] = df_11['timestamp'].dt.strftime('%d/%m/%Y %H:%M')
df_11["day"] = df_11['timestamp'].dt.strftime('%d/%m/%Y')
df_11["hour"] = df_11['timestamp'].dt.hour



In [24]:
df_callsigns

,ICAO,CS,FLIGHT_ID,MIN_TIMESTAMP,MAX_TIMESTAMP
0,000001,TALUD28_,0,2024-12-04 16:26:30.413,2024-12-04 16:32:29.696
1,000001,TALUD28_,1,2024-12-04 19:11:34.061,2024-12-04 19:19:52.507
2,000001,TALUD34_,0,2024-12-04 11:55:34.937,2024-12-04 12:08:57.597
3,010078,EGY1124_,0,2024-12-03 08:16:42.055,2024-12-03 08:19:32.103
4,0101ba,MSR754__,0,2024-12-01 13:21:31.690,2024-12-01 13:22:51.499
...,...,...,...,...,...
6952,e8046d,TAM8067_,0,2024-12-05 13:07:03.639,2024-12-05 13:07:03.639
6953,e8046d,TAM8068_,0,2024-12-05 06:30:44.882,2024-12-05 06:39:54.825
6954,e8046f,LAN9957_,0,2024-12-06 18:01:08.563,2024-12-06 18:06:32.385
6955,e80473,LAN714__,0,2024-12-02 12:10:37.432,2024-12-02 12:19:13.368


In [25]:
df_11

,index,ts_kafka,message,messageHex,DL,ICAO,OnGround,timestamp,formatted_date,day,hour
0,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0
1,28,1733011203261,XVAtXkh/IA==,5d502d5e487f20,11,502d5e,0.0,2024-12-01 00:00:03.261,01/12/2024 00:00,01/12/2024,0
3,56,1733011206603,XUytpDAQpw==,5d4cada43010a7,11,4cada4,0.0,2024-12-01 00:00:06.603,01/12/2024 00:00,01/12/2024,0
4,64,1733011206603,XUnS7vEibg==,5d49d2eef1226e,11,49d2ee,0.0,2024-12-01 00:00:06.603,01/12/2024 00:00,01/12/2024,0
5,70,1733011207114,XTRkDrEgdQ==,5d34640eb12075,11,34640e,0.0,2024-12-01 00:00:07.114,01/12/2024 00:00,01/12/2024,0
...,...,...,...,...,...,...,...,...,...,...,...
506046,5299964,1733125070047,XUcfa4h8qQ==,5d471f6b887ca9,11,471f6b,0.0,2024-12-02 07:37:50.047,02/12/2024 07:37,02/12/2024,7
506048,5299968,1733125070047,XUlSx/Z3nw==,5d4952c7f6779f,11,4952c7,0.0,2024-12-02 07:37:50.047,02/12/2024 07:37,02/12/2024,7
506049,5299974,1733125070047,XUlSEfI2yg==,5d495211f236ca,11,495211,0.0,2024-12-02 07:37:50.047,02/12/2024 07:37,02/12/2024,7
506050,5299987,1733125070354,XUlRzhvleg==,5d4951ce1be57a,11,4951ce,0.0,2024-12-02 07:37:50.354,02/12/2024 07:37,02/12/2024,7


In [26]:
df_merged = df_11.merge(df_callsigns,on="ICAO")
df_merged.head()


,index,ts_kafka,message,messageHex,DL,ICAO,OnGround,timestamp,formatted_date,day,hour,CS,FLIGHT_ID,MIN_TIMESTAMP,MAX_TIMESTAMP
0,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0,ANE010__,0,2024-12-02 16:31:08.981,2024-12-02 16:31:08.981
1,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0,ANE1357_,0,2024-12-06 15:40:09.139,2024-12-06 15:49:34.146
2,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0,ANE17NR_,0,2024-12-04 19:11:28.350,2024-12-04 19:18:58.456
3,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0,ANE3016_,0,2024-12-01 00:00:08.331,2024-12-01 00:09:00.104
4,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0,ANE74KE_,0,2024-12-04 07:20:27.918,2024-12-04 07:28:35.168


In [27]:
df_filtered = df_merged[(df_merged['timestamp'] >= df_merged['MIN_TIMESTAMP']) & 
                           (df_merged['timestamp'] <= df_merged['MAX_TIMESTAMP'])]

# Show the filtered dataframe
df_filtered

,index,ts_kafka,message,messageHex,DL,ICAO,OnGround,timestamp,formatted_date,day,hour,CS,FLIGHT_ID,MIN_TIMESTAMP,MAX_TIMESTAMP
10,302,1733011220497,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:20.497,01/12/2024 00:00,01/12/2024,0,ANE3016_,0,2024-12-01 00:00:08.331,2024-12-01 00:09:00.104
17,404,1733011226370,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:26.370,01/12/2024 00:00,01/12/2024,0,ANE3016_,0,2024-12-01 00:00:08.331,2024-12-01 00:09:00.104
24,513,1733011230732,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:30.732,01/12/2024 00:00,01/12/2024,0,ANE3016_,0,2024-12-01 00:00:08.331,2024-12-01 00:09:00.104
31,1399,1733011289658,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:01:29.658,01/12/2024 00:01,01/12/2024,0,ANE3016_,0,2024-12-01 00:00:08.331,2024-12-01 00:09:00.104
38,1602,1733011304564,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:01:44.564,01/12/2024 00:01,01/12/2024,0,ANE3016_,0,2024-12-01 00:00:08.331,2024-12-01 00:09:00.104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498101,5231700,1733125058474,XU0kk0REDg==,5d4d249344440e,11,4d2493,0.0,2024-12-02 07:37:38.474,02/12/2024 07:37,02/12/2024,7,WMT575__,0,2024-12-02 07:36:41.688,2024-12-02 07:38:11.678
2498102,5232683,1733125070760,XU0kk0REDg==,5d4d249344440e,11,4d2493,0.0,2024-12-02 07:37:50.760,02/12/2024 07:37,02/12/2024,7,WMT575__,0,2024-12-02 07:36:41.688,2024-12-02 07:38:11.678
2498103,5264194,1733125031737,XU0kk0REDg==,5d4d249344440e,11,4d2493,0.0,2024-12-02 07:37:11.737,02/12/2024 07:37,02/12/2024,7,WMT575__,0,2024-12-02 07:36:41.688,2024-12-02 07:38:11.678
2498104,5264249,1733125034099,XU0kk0REDg==,5d4d249344440e,11,4d2493,0.0,2024-12-02 07:37:14.099,02/12/2024 07:37,02/12/2024,7,WMT575__,0,2024-12-02 07:36:41.688,2024-12-02 07:38:11.678


In [28]:
df_11


,index,ts_kafka,message,messageHex,DL,ICAO,OnGround,timestamp,formatted_date,day,hour
0,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0
1,28,1733011203261,XVAtXkh/IA==,5d502d5e487f20,11,502d5e,0.0,2024-12-01 00:00:03.261,01/12/2024 00:00,01/12/2024,0
3,56,1733011206603,XUytpDAQpw==,5d4cada43010a7,11,4cada4,0.0,2024-12-01 00:00:06.603,01/12/2024 00:00,01/12/2024,0
4,64,1733011206603,XUnS7vEibg==,5d49d2eef1226e,11,49d2ee,0.0,2024-12-01 00:00:06.603,01/12/2024 00:00,01/12/2024,0
5,70,1733011207114,XTRkDrEgdQ==,5d34640eb12075,11,34640e,0.0,2024-12-01 00:00:07.114,01/12/2024 00:00,01/12/2024,0
...,...,...,...,...,...,...,...,...,...,...,...
506046,5299964,1733125070047,XUcfa4h8qQ==,5d471f6b887ca9,11,471f6b,0.0,2024-12-02 07:37:50.047,02/12/2024 07:37,02/12/2024,7
506048,5299968,1733125070047,XUlSx/Z3nw==,5d4952c7f6779f,11,4952c7,0.0,2024-12-02 07:37:50.047,02/12/2024 07:37,02/12/2024,7
506049,5299974,1733125070047,XUlSEfI2yg==,5d495211f236ca,11,495211,0.0,2024-12-02 07:37:50.047,02/12/2024 07:37,02/12/2024,7
506050,5299987,1733125070354,XUlRzhvleg==,5d4951ce1be57a,11,4951ce,0.0,2024-12-02 07:37:50.354,02/12/2024 07:37,02/12/2024,7


In [29]:
df_11[df_11["ICAO"]== "344213"]


,index,ts_kafka,message,messageHex,DL,ICAO,OnGround,timestamp,formatted_date,day,hour
0,4,1733011203260,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:03.260,01/12/2024 00:00,01/12/2024,0
20,302,1733011220497,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:20.497,01/12/2024 00:00,01/12/2024,0
29,404,1733011226370,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:26.370,01/12/2024 00:00,01/12/2024,0
32,513,1733011230732,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:00:30.732,01/12/2024 00:00,01/12/2024,0
83,1399,1733011289658,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-01 00:01:29.658,01/12/2024 00:01,01/12/2024,0
...,...,...,...,...,...,...,...,...,...,...,...
364771,3796238,1733500087329,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-06 15:48:07.329,06/12/2024 15:48,06/12/2024,15
364815,3796703,1733500091336,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-06 15:48:11.336,06/12/2024 15:48,06/12/2024,15
365007,3798514,1733500116083,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-06 15:48:36.083,06/12/2024 15:48,06/12/2024,15
365053,3799318,1733500128391,XTRCE1QV9w==,5d3442135415f7,11,344213,0.0,2024-12-06 15:48:48.391,06/12/2024 15:48,06/12/2024,15


In [30]:
df_11_grouped = df_11.groupby(["day", "hour", "ICAO"])["OnGround"].nunique().reset_index()
df_11_grouped.sort_values(by ="OnGround", ascending=False)

,day,hour,ICAO,OnGround
4571,06/12/2024,11,3452c6,2
3712,05/12/2024,11,355316,2
3192,04/12/2024,14,502c22,2
4866,06/12/2024,15,3451da,2
2273,03/12/2024,15,34368c,2
...,...,...,...,...
2007,03/12/2024,10,345681,1
2006,03/12/2024,10,345251,1
2005,03/12/2024,10,344697,1
2004,03/12/2024,10,3445d4,1


In [32]:
df_11_grouped = df_11.groupby(["day","hour","ICAO"])["OnGround"].unique().explode().reset_index()
df_11_grouped

,day,hour,ICAO,OnGround
0,01/12/2024,0,02012b,0.0
1,01/12/2024,0,020176,0.0
2,01/12/2024,0,342352,1.0
3,01/12/2024,0,342353,1.0
4,01/12/2024,0,344115,1.0
...,...,...,...,...
6034,07/12/2024,20,4d24c4,0.0
6035,07/12/2024,20,4d2535,0.0
6036,07/12/2024,20,781ff7,0.0
6037,07/12/2024,20,89631d,0.0


In [33]:
days = df_11_grouped["day"].unique()
df = df_11_grouped[df_11_grouped["day"] ==days[0]]
df["OnGround"].value_counts()

0.0    942
1.0     31
Name: OnGround, dtype: int64

In [34]:
days = df_11_grouped["day"].unique()
df = df_11_grouped[df_11_grouped["day"] ==days[0]]
df.groupby(["hour","OnGround"])["OnGround"].size().reset_index(name="count")

,hour,OnGround,count
0,0,0.0,19
1,0,1.0,4
2,7,0.0,106
3,7,1.0,3
4,9,0.0,166
5,9,1.0,4
6,11,0.0,145
7,11,1.0,4
8,13,0.0,132
9,13,1.0,5


In [42]:
df["OnGround"] = df["OnGround"].replace({1: "On Ground", 0: "In Air"})
df

/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



,hour,OnGround,count
0,0,In Air,5
1,0,On Ground,5
2,1,In Air,4
3,1,On Ground,4
4,7,In Air,120
5,7,On Ground,4
6,9,In Air,110
7,9,On Ground,5
8,11,In Air,130
9,11,On Ground,4


In [46]:
fig.update_xaxes?

Signature:
fig.update_xaxes(
    patch=None,
    selector=None,
    overwrite=False,
    row=None,
    col=None,
    **kwargs,
) -> 'Figure'
Docstring:
Perform a property update operation on all xaxis objects
that satisfy the specified selection criteria

Parameters
----------
patch: dict
    Dictionary of property updates to be applied to all
    xaxis objects that satisfy the selection criteria.
selector: dict, function, or None (default None)
    Dict to use as selection criteria.
    xaxis objects will be selected if they contain
    properties corresponding to all of the dictionary's keys, with
    values that exactly match the supplied values. If None
    (the default), all xaxis objects are selected. If a
    function, it must be a function accepting a single argument and
    returning a boolean. The function will be called on each
    xaxis and those for which the function returned True will
    be in the selection.
overwrite: bool
    If True, overwrite existing properties. If

In [59]:
df_full

,hour,OnGround,count
0,1,In Air,4.0
1,1,On Ground,4.0
2,2,NaN,3.0
3,3,NaN,3.0
4,4,NaN,3.0
5,5,NaN,3.0
6,6,NaN,3.0
7,7,In Air,120.0
8,7,On Ground,4.0
9,8,NaN,3.0


In [60]:
px.bar(df_full, x="hour", y="count", color="OnGround", title=f"Day {day}")

In [ ]:
import plotly.express as px
days = df_11_grouped["day"].unique()
for day in days:
    df = df_11_grouped[df_11_grouped["day"] ==day]
    df =df.groupby(["hour","OnGround"])["OnGround"].size().reset_index(name="count")
    df["OnGround"] = df["OnGround"].replace({1: "On Ground", 0: "In Air"})
    
    fig = px.bar(df, x="hour", y="count", color="OnGround", title=f"Day {day}")

    fig.show()
    


/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



/Users/hamzatriki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

